In [0]:
from pyspark.sql.types import (
    StructType, StructField, StringType, LongType
)

from pyspark.sql.types import *

json_schema = StructType([
    StructField("Body", StructType([
        StructField("mnsn", StringType(), True),
        StructField("scheduleName", StringType(), True),

        StructField("smsProperties", StructType([
            StructField("device", StructType([
                StructField("address", StringType(), True),

                StructField("disposalMarkerList", ArrayType(
                    StructType([
                        StructField("amount", StructType([
                            StructField("capacity", StringType(), True),
                            StructField("state", StringType(), True),
                            StructField("typical", StringType(), True),
                            StructField("unit", StringType(), True)
                        ]), True),
                        StructField("color", StringType(), True),
                        StructField("description", StringType(), True),
                        StructField("type", StringType(), True)
                    ])
                ), True),

                StructField("familyName", StringType(), True),
                StructField("friendlyName", StringType(), True),

                StructField("inTrayList", ArrayType(
                    StructType([
                        StructField("amount", StructType([
                            StructField("capacity", StringType(), True),
                            StructField("state", StringType(), True),
                            StructField("typical", StringType(), True),
                            StructField("unit", StringType(), True)
                        ]), True),
                        StructField("description", StringType(), True),
                        StructField("id", StringType(), True),
                        StructField("inserter", StringType(), True),
                        StructField("manual", StringType(), True),
                        StructField("mediaDimFeed", StringType(), True),
                        StructField("mediaDimXFeed", StringType(), True),
                        StructField("mediaName", StringType(), True),
                        StructField("mediaSize", StringType(), True),
                        StructField("mediaSizeHeight", StringType(), True),
                        StructField("mediaSizeName", StringType(), True),
                        StructField("mediaSizeUnit", StringType(), True),
                        StructField("mediaSizeWidth", StringType(), True),
                        StructField("mediaType", StringType(), True),
                        StructField("modelName", StringType(), True),
                        StructField("name", StringType(), True),
                        StructField("virtual", StringType(), True)
                    ])
                ), True),

                StructField("location", StructType([
                    StructField("address", StringType(), True)
                ]), True),

                StructField("markerList", ArrayType(
                    StructType([
                        StructField("amount", StructType([
                            StructField("capacity", StringType(), True),
                            StructField("state", StringType(), True),
                            StructField("typical", StringType(), True),
                            StructField("unit", StringType(), True)
                        ]), True),
                        StructField("color", StringType(), True),
                        StructField("description", StringType(), True),
                        StructField("type", StringType(), True)
                    ])
                ), True),

                StructField("modelName", StringType(), True),

                StructField("outTrayList", ArrayType(
                    StructType([
                        StructField("amount", StructType([
                            StructField("capacity", StringType(), True),
                            StructField("state", StringType(), True),
                            StructField("typical", StringType(), True),
                            StructField("unit", StringType(), True)
                        ]), True),
                        StructField("deliveryOrientation", StringType(), True),
                        StructField("description", StringType(), True),
                        StructField("id", StringType(), True),
                        StructField("modelName", StringType(), True),
                        StructField("name", StringType(), True),
                        StructField("stackingOrder", StringType(), True)
                    ])
                ), True),

                StructField("serialId", StringType(), True),
                StructField("statusRawValue", StringType(), True),

                StructField("suppliesCounter", StructType([
                    StructField("TYPE", StructType([
                        StructField("toner", StructType([
                            StructField("large", StructType([
                                StructField("end", StructType([
                                    StructField("black", StructType([StructField("value", LongType(), True)]), True),
                                    StructField("cyan", StructType([StructField("value", LongType(), True)]), True),
                                    StructField("magenta", StructType([StructField("value", LongType(), True)]), True),
                                    StructField("yellow", StructType([StructField("value", LongType(), True)]), True)
                                ]), True)
                            ]), True)
                        ]), True)
                    ]), True)
                ]), True)

            ]), True)
        ]), True),

        StructField("interface", StructType([
            StructField("ethernetList", ArrayType(
                StructType([
                    StructField("address", StringType(), True),
                    StructField("id", StringType(), True),
                    StructField("type", StringType(), True)
                ])
            ), True),
            StructField("ipList", ArrayType(
                StructType([
                    StructField("address", StringType(), True),
                    StructField("defaultRoute", StringType(), True),
                    StructField("ethernetId", StringType(), True),
                    StructField("subnetMask", StringType(), True)
                ])
            ), True)
        ]), True),

        StructField("timestamp", LongType(), True),
        StructField("type", StringType(), True)
    ]), True),

    StructField("EnqueuedTimeUtc", StringType(), True),

    StructField("Properties", StructType([
        StructField("appTopic", StringType(), True),
        StructField("customerId", StringType(), True),
        StructField("dealerId", StringType(), True),
        StructField("relatedGroupId", StringType(), True)
    ]), True),

    StructField("SystemProperties", StructType([
        StructField("connectionAuthMethod", StringType(), True),
        StructField("connectionDeviceGenerationId", StringType(), True),
        StructField("connectionDeviceId", StringType(), True),
        StructField("contentEncoding", StringType(), True),
        StructField("contentType", StringType(), True),
        StructField("enqueuedTime", StringType(), True)
    ]), True)
])

# display(json_schema)

df = spark.read.format("json").schema(json_schema).load('/Volumes/workspace/default/json_data/15/')
# df = spark.read.json('/Volumes/workspace/default/json_data/15/02-28.json')

display(df)

In [0]:
schema = df.printSchema()

In [0]:
from pyspark.sql.functions import col, struct
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# ===============================
# Transform + Re-nest + Dedup
# ===============================

result_df = (
    df
    # -------- FLATTEN --------
    .select(
        col("Body.mnsn").alias("mnsn"),
        col("Body.timestamp").alias("timestamp"),
        col("Body.type").alias("type"),

        col("Properties.relatedGroupId").alias("relatedGroupId"),
        col("Properties.customerId").alias("customerId"),
        col("Properties.dealerId").alias("dealerId"),

        col("Body.smsProperties.device.serialId").alias("serialNumber"),
        col("Body.smsProperties.device.modelName").alias("modelName"),
        col("Body.smsProperties.device.familyName").alias("familyName"),
        col("Body.smsProperties.device.friendlyName").alias("friendlyName"),
        col("Body.smsProperties.device.statusRawValue").alias("statusCode"),
        col("Body.smsProperties.device.address").alias("location"),

        col("Body.interface.ethernetList").alias("ethernetList"),
        col("Body.interface.ipList").alias("ipList"),

        col("Body.smsProperties.device.inTrayList").alias("inTrayList"),
        col("Body.smsProperties.device.outTrayList").alias("outTrayList"),
        col("Body.smsProperties.device.markerList").alias("markerList"),
        col("Body.smsProperties.device.disposalMarkerList").alias("disposalMarkerList"),

        col("Body.smsProperties.device.suppliesCounter.TYPE.toner.large.end.black.value").alias("toner_black"),
        col("Body.smsProperties.device.suppliesCounter.TYPE.toner.large.end.cyan.value").alias("toner_cyan"),
        col("Body.smsProperties.device.suppliesCounter.TYPE.toner.large.end.magenta.value").alias("toner_magenta"),
        col("Body.smsProperties.device.suppliesCounter.TYPE.toner.large.end.yellow.value").alias("toner_yellow")
    )

    # -------- RE-NEST (Delta schema) --------
    .select(
        col("mnsn"),
        col("timestamp").alias("createTimestamp"),
        col("type"),
        col("relatedGroupId"),
        col("customerId"),
        col("dealerId"),

        struct(
            col("serialNumber"),
            col("modelName"),
            col("familyName"),
            col("friendlyName"),
            col("statusCode"),
            col("location")
        ).alias("deviceGeneral"),

        struct(
            col("ethernetList"),
            col("ipList")
        ).alias("interface"),

        col("inTrayList"),
        col("outTrayList"),
        col("markerList"),
        col("disposalMarkerList"),

        struct(
            struct(
                struct(
                    struct(
                        struct(
                            struct(col("toner_black").alias("value")).alias("black"),
                            struct(col("toner_cyan").alias("value")).alias("cyan"),
                            struct(col("toner_magenta").alias("value")).alias("magenta"),
                            struct(col("toner_yellow").alias("value")).alias("yellow")
                        ).alias("end")
                    ).alias("large")
                ).alias("toner")
            ).alias("TYPE")
        ).alias("counter")
    )

    # -------- DEDUP --------
    .withColumn(
        "rn",
        row_number().over(
            Window.partitionBy("mnsn")
                  .orderBy(col("createTimestamp").desc())
        )
    )
    .filter(col("rn") == 1)
    .drop("rn")
)

display(result_df)


In [0]:
# from pyspark.sql.functions import col
    # .option("mergeSchema", "true") \

# result_df.write \
#     .format("delta") \
#     .option("mergeSchema", "true") \
#     .mode("overwrite") \
#     .saveAsTable("workspace.default.iot_device_events")



In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(
    spark,
    "workspace.default.iot_device_events"
)

deltaTable.alias("table") \
  .merge(
      result_df.alias("res"),
      "table.mnsn = res.mnsn"
  ) \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()

